In [1]:
# подключение библиотек
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as sts
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.svm import SVC
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
plt.style.use('ggplot')
from matplotlib.pyplot import figure
from IPython.core.pylabtools import figsize
import itertools
from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12,8)

pd.options.mode.chained_assignment = None

In [13]:
# загрузка выборки
df=pd.read_csv('input.csv')

In [14]:
# просмотр
df

,Hemoglobin,Erythrocytes,Color_index,Leukocytes,Neutrophils,Neutrophils_segmented,Lymphocytes,Monocytes,Erythrocyte,Total_bilirubin,Total_protein,Amylase,Glucose,Duration,sex,conclusion
0,146.0,4.5,0.97,7.6,9,65,21,5,15,12.8,67.00,18.2,5.8,150,1,0
1,141.0,4.5,0.94,8.7,2,70,27,1,8,14.0,66.00,14.0,5.0,40,1,0
2,162.0,4.8,1.00,8.1,6,80,12,3,15,13.4,77.00,16.8,8.7,55,1,0
3,162.1,4.8,1.00,9.0,12,78,5,5,15,16.3,72.00,30.1,5.8,50,1,0
4,142.8,4.6,0.93,12.5,6,48,43,2,18,8.5,63.00,21.0,4.6,70,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,134.0,4.4,0.92,5.0,5,54,36,5,35,50.0,69.00,19.4,62.0,120,1,1
105,148.6,4.6,0.97,11.4,6,60,32,1,15,68.0,68.00,142.2,6.8,35,1,1
106,118.0,4.3,0.89,9.9,8,65,22,4,45,15.0,68.00,27.0,5.3,45,0,1
107,142.8,4.6,0.93,10.3,26,50,23,1,40,29.5,79.00,27.0,6.2,110,0,1


In [15]:
def preproc(df):
# загрузим данные
#будет установлена первая строка в качестве имен столбцов dataframe
    #df.columns = df.iloc[0]
    #удаляем 0-строку
   # df=df.drop(df.index[[0,2]])
    #преобразуем к числовому формату
    df=df.iloc[0:108].apply(pd.to_numeric)
    x=df.iloc[:,0:15]
    y=df.iloc[:,-1]
    print(x)
    print(y)
    return df,x,y

In [ ]:
def stat(df):
    # описательная статистика для числовых данных
    print(df.describe())
    a = []
    b = []
    cols = list(df)
    for i in cols:
        kurt = sts.kurtosis(df[i], axis=0, fisher=True, bias=True)
        sk = sts.skew(df[i], axis=0, bias=True)
        print('Показатели ассиметрии и эксцесса для: {}'.format(i))
        print(kurt, sk)
        a.append(kurt)
        b.append(sk)
    return a, b

In [ ]:
def hitg(df):
    fig = plt.figure(figsize=(15, 20))
    ax = fig.gca()
    df.hist(ax=ax)
    #plt.title('Гистограммы показателей',fontsize=10)
    #plt.xlabel('Интервалы распределения',fontsize=10)
    #plt.ylabel('Частота',fontsize=10)
    plt.show()

    sns.heatmap(df.corr(), annot=True, cmap='RdYlGn', linewidths=0.2)
    fig = plt.gcf()
    fig.set_size_inches(12, 12)
    plt.show()

In [ ]:
def k_best(x, y):
    bestfeature = SelectKBest(score_func=chi2, k='all')
    fit = bestfeature.fit(x, y)
    df_score = pd.DataFrame(fit.scores_)
    df_column = pd.DataFrame(x.columns)
    feature_score = pd.concat([df_column, df_score], axis=1)
    feature_score.columns = ['Specs', 'Score']
    feature_score = feature_score.sort_values(by='Score', ascending=False)
    print('Значимость показателей')
    print(feature_score)
    # выбираем показатели по важности
    f_s1 = feature_score[feature_score['Score'] > 20]
    drop_list1 = f_s1['Specs']
    #drop_list1.head(7)
    #feature_score
    x_1 = x.loc[:, drop_list1]  #3
    print(
        'Используя метод фльтрации Хи2 для отбра показателей мы получили следующий результат. Наиболее информативными показателями являются:')
    print(drop_list1)
    print('Создадим новую выборку состоящую толко из этих показателей.')
    print(x_1)
    return feature_score, x_1, f_s1

In [ ]:
def forest_select(x, y):
    clf_rf_3 = RandomForestClassifier(criterion='entropy', max_depth=2, bootstrap='bool', random_state=1)
    clf_rf_2 = AdaBoostClassifier()
    #mlp = MLPClassifier(activation='relu', solver='lbfgs', alpha=1e-5, max_iter=1000, hidden_layer_sizes=(3, ), random_state=1)
    #mlp_full=mlp.fit(x_full_train, y_train)
    rfe = RFECV(estimator=clf_rf_3, step=1, cv=5)
    rfe1 = RFE(estimator=clf_rf_3, n_features_to_select=5, step=1)
    # fff1=rfe.fit(x, y)
    ffff2 = rfe1.fit(x, y)
    # нужно проверить
    print(f'вектор. {rfe1.support_}')
    print('Ранжирование объектов с рекурсивным исключением с использование метода RandomForest::',
          x.columns[rfe1.support_])
    # формирование новой выборки
    x_12 = x.loc[:, x.columns[rfe1.support_]]
    return x_12

In [16]:
df,x,y=preproc(df)

     Hemoglobin  Erythrocytes  Color_index  Leukocytes   Neutrophils  \
0         146.0           4.5         0.97         7.6             9   
1         141.0           4.5         0.94         8.7             2   
2         162.0           4.8         1.00         8.1             6   
3         162.1           4.8         1.00         9.0            12   
4         142.8           4.6         0.93        12.5             6   
..          ...           ...          ...         ...           ...   
103       138.0           4.5         0.92        18.6            18   
104       134.0           4.4         0.92         5.0             5   
105       148.6           4.6         0.97        11.4             6   
106       118.0           4.3         0.89         9.9             8   
107       142.8           4.6         0.93        10.3            26   

      Neutrophils_segmented  Lymphocytes  Monocytes  Erythrocyte  \
0                        65           21          5           15   